In [1]:
%matplotlib inline

In [21]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
plt.style.use('ggplot')

from sklearn.neighbors import DistanceMetric
from math import radians
import re
from googletrans import Translator
import plotly.express as px

In [7]:
# Read the directory.csv file into a data frame
starbucks = pd.read_csv('directory.csv')
starbucks= starbucks[['store_name','City', 'State/Province','Country','Longitude', 'Latitude']]


#Create a new column for store count
starbucks['store_count'] = np.ones(starbucks.shape[0]).astype(int).tolist()
starbucks.sample(5)

#Read the country code file into the dateframe.
country_code = pd.read_csv('country_code.csv')
country_code

#Dataframe for the world population
world_population = pd.read_csv('world_population.csv')
world_population = world_population[['country', 'population']]
world_population.loc[world_population.country=='United States']

#DataFrame for the gdp
gdp=pd.read_csv('gdp.csv')

starbucks.sample(6)


,store_name,City,State/Province,Country,Longitude,Latitude,store_count
22624,Pottstown,Stowe,PA,US,-75.65,40.26,1
2502,哈尔滨建设街咖啡店,哈尔滨,23,CN,126.64,45.76,1
2661,古北店,上海市,31,CN,121.39,31.19,1
13216,Felicita & Centre City Parkway,Escondido,CA,US,-117.08,33.10,1
4008,武汉街道口新世界店,武汉市,42,CN,114.35,30.53,1
2539,黄浦永新广场店,上海市,31,CN,121.47,31.23,1


In [8]:
#Merge the starbucks data frame and the country code
starbucks2= pd.merge(starbucks,country_code, left_on='Country', right_on='Two_Letter_Country_Code').drop('Two_Letter_Country_Code', axis=1)
starbucks2.sample(5)

,store_name,City,State/Province,Country,Longitude,Latitude,store_count,Continent_Name,Country_Name
20674,Hy Vee - Omaha #1471,Omaha,NE,US,-96.12,41.27,1,North America,United States of America
11474,Starbucks,Ankara,6,TR,32.83,39.85,1,Europe,"Turkey, Republic of"
22319,Tylersville & Cox,Westchester,OH,US,-84.37,39.35,1,North America,United States of America
22801,Collier Town Square,Bridgeville,PA,US,-80.10,40.38,1,North America,United States of America
12046,Xi Da,Hsinchu City,HSZ,TW,121.02,24.82,1,Asia,Taiwan


In [9]:
#Modify the name of some countries, removing 'the, republic of'
country_name = starbucks2['Country_Name'].tolist()

country_name = [country.split(',')[0] for country in country_name]

#Assign the modified values in the country_name column
starbucks2['Country_Name'] = country_name


#Change united states of america to united states
starbucks2['Country_Name']= starbucks2['Country_Name'].apply(lambda x : 'United States' if x=='United States of America' else x)

#Change Russian Federation to Russia
starbucks2['Country_Name']= starbucks2['Country_Name'].apply(lambda x : 'Russia' if x=='Russian Federation' else x)


starbucks2['Country_Name']= starbucks2['Country_Name'].apply(lambda x : 'South Korea' if x=='Korea' else x)

#Change Russian Federation to Russia
starbucks2['Country_Name']= starbucks2['Country_Name'].apply(lambda x : 'Czech Republic' if x=='Czech Republic (Czechia)' else x)


# Change column names to lower case
starbucks2.columns = [x.lower()  for x in starbucks2.columns]
starbucks2.sample(5)


,store_name,city,state/province,country,longitude,latitude,store_count,continent_name,country_name
24932,Gaskins & Mayland,Richmond,VA,US,-77.57,37.64,1,North America,United States
13790,"Peach & McKinley, Fresno",Fresno,CA,US,-119.72,36.76,1,North America,United States
18278,Butterfield & Meyers,Lombard,IL,US,-88.00,41.84,1,North America,United States
8519,Incheon Yeonsu,Incheon,28,KR,126.68,37.42,1,Asia,South Korea
25482,4637 Sunset Blvd.,Renton,WA,US,-122.16,47.51,1,North America,United States


In [11]:
#Merge world population data with starbucks data 
starbucks3 = pd.merge(starbucks2, world_population, how='left', left_on='country_name', right_on='country')
starbucks3.drop('country_y', axis=1, inplace=True)

#Rename some columns
starbucks3.rename(columns={'country_x':'code', 'continent_name':'continent', 'country_name':'country'}, inplace=True)
starbucks3.sample(5)


(26057, 10)

In [20]:
#Merge gdp data with starbucks data 
starbucks4 = pd.merge(starbucks3, gdp, how='left', left_on='country', right_on='country')
starbucks4.drop('code_x', axis=1, inplace=True)
starbucks4.rename(columns={'code_y': 'code'}, inplace=True)
starbucks4.sample(5)


,store_name,city,state/province,longitude,latitude,store_count,continent,country,population,code,gdp_ppp
5661,Earls Court Road,London,ENG,-0.19,51.49,1,Europe,United Kingdom,"67,886,011",GBR,3.260000e+12
22977,Target Philadelphia NE T-1809,Philadelphia,PA,-75.01,40.10,1,North America,United States,"331,002,651",USA,2.140000e+13
6920,Kawaguchi,Kawaguchi,11,139.72,35.80,1,Asia,Japan,"126,476,461",JPN,5.460000e+12
1440,Longos Applewood plaza,Mississauga,ON,-79.58,43.59,1,North America,Canada,"37,742,154",CAN,1.930000e+12
16878,Beach & University,Jacksonville,FL,-81.60,30.29,1,North America,United States,"331,002,651",USA,2.140000e+13


In [8]:
#Top 15 countries in terms of starbucks store count
df_bar= starbucks4[['country','store_count']].groupby('country').sum().sort_values( ascending=False, by='store_count').reset_index().head(15)



In [27]:
#Create a dataframe with the most starbucks store in the world
df_bar= starbucks4[['country','store_count']].groupby('country').sum().sort_values( ascending=False, by='store_count').reset_index().head(15)

#Display a bar graph
fig = px.bar(df_bar, x='country', y ='store_count')
fig.show()

In [28]:
# #Remove the comma in the population 78,100 -->78100
pop_df= starbucks4.copy()
pop_df.dropna(axis=0, inplace=True)
pop_df['population']= pop_df['population'].str.replace(',', '').astype(float)


In [29]:
#Create a new data frame to include modified population values
df_scatter= pop_df.groupby(['country', 'population', 'continent', 'gdp_ppp']).agg({'store_count':'sum'}).reset_index()
df_scatter.sample(5)

,country,population,continent,gdp_ppp,store_count
60,Switzerland,8654622.0,Europe,6.090000e+11,61
46,Philippines,109581078.0,Asia,1.000000e+12,298
31,Jordan,10203134.0,Asia,1.040000e+11,17
6,Bahrain,1701575.0,Asia,7.695761e+10,21
23,Germany,83783942.0,Europe,4.660000e+12,160


In [30]:
#Plot a scatter plot with population , gdp and store count
df = df_scatter
fig = px.scatter(df, x="population", y="gdp_ppp", size="store_count",color='continent',
           hover_name="country", log_x=True, size_max=50)
fig.show()



In [31]:
#Create a new data frame without us, china and india
df_scatter2=df_scatter.loc[(df_scatter.country !='United States') & (df_scatter.country !='China') & (df_scatter.country !='India')]

#Graph the new dataframe
df = df_scatter2
fig = px.scatter(df, x="population", y="gdp_ppp", size="store_count",color='continent',
           hover_name="country", log_x=True, size_max=20)
fig.show()



In [83]:
city_df = starbucks3.loc[starbucks4.gdp_ppp>5e9]
city_df = city_df.dropna(axis=0)
city_df.set_index('city', inplace=True)
city_df.rename(index={'上海市':'Shanghai', '北京市':'Beijing','서울':'Seoul', '杭州市':'Hangzhou'}, inplace=True)
city_df.shape



(25584, 9)

In [84]:
city_df.reset_index()
city_bar = city_df.groupby(['city', 'country', 'continent']).agg({'store_count':'sum'}).sort_values(by='store_count',ascending=False)
city_bar

,,,store_count
city,country,continent,
Shanghai,China,Asia,544
Seoul,South Korea,Asia,378
Beijing,China,Asia,234
New York,United States,North America,232
London,United Kingdom,Europe,195
...,...,...,...
Lake Placid,United States,North America,1
Lake Ridge,United States,North America,1
Lake St. Louis,United States,North America,1


In [25]:
def avg_dist(df):
    
    """
    df: is a pandas dataframe

    Return the average distance of stores in miles
    """

    #Convert lat and lon degrees in radians
    df['latitude'] = np.radians(df['latitude'])
    df['longitude'] = np.radians(df['longitude'])

    #find the haversine distance
    dist = DistanceMetric.get_metric('haversine')

    #an array of distance of neighboring stores in miles
    array =dist.pairwise(df[['latitude','longitude']].to_numpy())*3798

    return np.mean(array[0])


In [85]:
city1 =city_bar.reset_index().loc[:,['city','country']].to_numpy()


In [30]:
city_df.reset_index()
city_list =[]
distance_list =[]

for ele in city1:
    city=ele[0]
    country= ele[1]
    
    city_2 =city_df.loc[(city_df.city==city) & (city_df.country==country)]
    distance = avg_dist(city2)
    
    city_list.append(city)
    distance_list.append(distance)
    


ValueError: cannot insert level_0, already exists

In [ ]:

distance_df =pd.DataFrame({'city': pd.Series(city_list), 'avg_dist':pd.Series(distance_list)})
distance_df
    